In [1]:
import random, gc, os, pickle, csv

import models.utils
from models.cls_oml_ori_v2 import OML

import higher
import numpy as np
import torch
import torch.nn.functional as F

In [2]:
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model_path1 = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path1 = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path1 = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
model_path1 = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"

# model_path2 = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path2 = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path2 = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
model_path2 = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"

# memory_path2 = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path2 = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path2 = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
memory_path2 = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"


In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*10,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*20,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600
}
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# 3.1 Find Answer Disparity

# 3.2 Find Diff

Good way to calculate element-wise difference between two models of the same structure
https://discuss.pytorch.org/t/good-way-to-calculate-element-wise-difference-between-two-models-of-the-same-structure/67592/2

In [5]:
def model_diff(model1, model2):
    return sum((x - y).abs().sum() for x, y in zip(model1.state_dict().values(), model2.state_dict().values())).item()

In [6]:
learner1 = OML(device=device, n_classes=n_classes, **args)
learner1.load_model(model_path1)

learner2 = OML(device=device, n_classes=n_classes, **args)
learner2.load_model(model_path2)
with open(memory_path2, 'rb') as f:
    learner2.memory = pickle.load(f)

2022-08-17 10:59:53,506 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2022-08-17 10:59:54,673 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2022-08-17 10:59:54,677 - transformers.configuration_utils - INFO - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

In [7]:
model_diff(learner1.rln, learner2.rln) + model_diff(learner1.pln, learner2.pln)

0.0

## Diff Adaptation

In [8]:
support_set = []
for _ in range(updates):
    text, labels = learner2.memory.read_batch(batch_size=mini_batch_size)
    support_set.append((text, labels))

with higher.innerloop_ctx(learner2.pln, learner2.inner_optimizer,
                          copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
    # Inner loop
    task_predictions, task_labels = [], []
    support_loss = []
    for text, labels in support_set:
        labels = torch.tensor(labels).to(device)
        input_dict = learner2.rln.encode_text(text)
        _repr = learner2.rln(input_dict)
        output = fpln(_repr)
        loss = learner2.loss_fn(output, labels)
        diffopt.step(loss)
        pred = models.utils.make_prediction(output.detach())
        support_loss.append(loss.item())
        task_predictions.extend(pred.tolist())
        task_labels.extend(labels.tolist())
    
    print( model_diff(learner1.pln, fpln))

23.940174102783203


# 3.3 Probing

How?